In [1]:
!pip install phonenumbers
import pandas as pd
import re
import phonenumbers

## Load Datasets

In [2]:
patients_data = pd.read_csv ('Patients_clean.csv')
provider_data = pd.read_csv ('provider_clean.csv')
sale_data = pd.read_csv ('sale_clean.csv')
dummy_data = pd.read_csv ('dummydata_clean.csv')

C:\Users\mchou\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
patients_data.head
provider_data.head
sale_data.head
provider_data

,Id,ORGANIZATION,NAME,GENDER,SPECIALITY,ADDRESS,CITY,STATE,ZIP,LAT,LON
0,4781b3e5-e13c-3ef0-945d-649abe044d47,ef58ea08-d883-3957-8300-150554edc8fb,Jeffrey Beer,M,GENERAL PRACTICE,60 HOSPITAL ROAD,LEOMINSTER,MA,01453,42.520838,-71.770876
1,a50f1afd-2a19-3f0f-a3f0-d9d8e51c6f9b,69176529-fd1f-3b3f-abce-a0a3626769eb,Allison Bednar,F,GENERAL PRACTICE,330 MOUNT AUBURN STREET,CAMBRIDGE,MA,02138,42.375967,-71.118275
2,692c8e0d-d680-3c70-9dfd-dfc2cacf3db5,5e765f2b-e908-3888-9fc7-df2cb87beb58,Angelena Torp,F,GENERAL PRACTICE,211 PARK STREET,ATTLEBORO,MA,02703,41.931653,-71.294503
3,f4f87207-b74e-390e-b1c9-ea000db107c3,f1fbcbfb-fcfa-3bd2-b7f4-df20f1b3c3a4,Denver Trantow,M,GENERAL PRACTICE,ONE GENERAL STREET,LAWRENCE,MA,01842,42.700273,-71.161357
4,75322072-9b0a-302a-b343-326fd18aeb1c,e002090d-4e92-300e-b41e-7d1f21dee4c6,Carlos Rath,M,GENERAL PRACTICE,1493 CAMBRIDGE STREET,CAMBRIDGE,MA,02138,42.375967,-71.118275
...,...,...,...,...,...,...,...,...,...,...,...
60529,17e4cdf8-33e0-3ddd-bec8-fd7375599e31,700056a6-3aea-3efd-8423-a9acbd38a5ea,Dudley Stehr,M,GENERAL PRACTICE,106 UNITED STATES HIGHWAY 44,RAYNHAM,MA,2767,41.905194,-71.060999
60530,9575422d-51ec-341b-90e9-ff3b4ecefe35,308b5efa-a9c2-3728-b1ff-d29c5f060731,Carlena Fisher,F,GENERAL PRACTICE,630 PLANTATION STREET,WORCESTER,MA,1605,42.292559,-71.760317
60531,9703163d-bbb7-3f3b-b7b6-9c065606ae57,bbf934c6-faad-3483-a1d2-d2ecb693a0e2,Bonnie Olson,F,GENERAL PRACTICE,309 TEATICKET HIGHWAY,TEATICKET,MA,2536,41.568344,-70.594006
60532,fd835e52-39d4-398b-bd62-89095d983cb2,90cf148c-69ed-3d33-aa86-b509c6b0b25f,Tayna Flatley,F,GENERAL PRACTICE,170 GOVERNORS AVENUE,MEDFORD,MA,2155,42.425909,-71.111365


## Extract the headers

In [5]:
patients_column_list=list(patients_data.columns.values)
patients_column_list

['Id',
 'BIRTHDATE',
 'SSN',
 'DRIVERS',
 'PASSPORT',
 'FIRST',
 'LAST',
 'MARITAL',
 'RACE',
 'ETHNICITY',
 'GENDER',
 'BIRTHPLACE',
 'ADDRESS',
 'CITY',
 'STATE',
 'COUNTY',
 'ZIP',
 'LAT',
 'LON',
 'HEALTHCARE_EXPENSES',
 'HEALTHCARE_COVERAGE',
 'full_name']

In [6]:
provider_column_list=list(provider_data.columns.values)
provider_column_list

['Id',
 'ORGANIZATION',
 'NAME',
 'GENDER',
 'SPECIALITY',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'LAT',
 'LON']

In [7]:
sale_column_list=list(sale_data.columns.values)
sale_column_list

['order_id',
 'order_date',
 'status',
 'item_id',
 'sku',
 'qty_ordered',
 'price',
 'value',
 'discount_amount',
 'total',
 'category',
 'payment_method',
 'bi_st',
 'cust_id',
 'year',
 'month',
 'ref_num',
 'Name Prefix',
 'First Name',
 'Middle Initial',
 'Last Name',
 'Gender',
 'age',
 'full_name',
 'E Mail',
 'Customer Since',
 'SSN',
 'Phone No. ',
 'Place Name',
 'County',
 'City',
 'State',
 'Zip',
 'Region',
 'User Name',
 'Discount_Percent']

In [8]:
dummy_column_list=list(dummy_data.columns.values)
dummy_column_list

['Full_Name', 'SSN', 'Credit_Card_Number']

## Create a list of lists

In [9]:
column_list = [patients_column_list,provider_column_list,sale_column_list,dummy_column_list] 
column_list

[['Id',
  'BIRTHDATE',
  'SSN',
  'DRIVERS',
  'PASSPORT',
  'FIRST',
  'LAST',
  'MARITAL',
  'RACE',
  'ETHNICITY',
  'GENDER',
  'BIRTHPLACE',
  'ADDRESS',
  'CITY',
  'STATE',
  'COUNTY',
  'ZIP',
  'LAT',
  'LON',
  'HEALTHCARE_EXPENSES',
  'HEALTHCARE_COVERAGE',
  'full_name'],
 ['Id',
  'ORGANIZATION',
  'NAME',
  'GENDER',
  'SPECIALITY',
  'ADDRESS',
  'CITY',
  'STATE',
  'ZIP',
  'LAT',
  'LON'],
 ['order_id',
  'order_date',
  'status',
  'item_id',
  'sku',
  'qty_ordered',
  'price',
  'value',
  'discount_amount',
  'total',
  'category',
  'payment_method',
  'bi_st',
  'cust_id',
  'year',
  'month',
  'ref_num',
  'Name Prefix',
  'First Name',
  'Middle Initial',
  'Last Name',
  'Gender',
  'age',
  'full_name',
  'E Mail',
  'Customer Since',
  'SSN',
  'Phone No. ',
  'Place Name',
  'County',
  'City',
  'State',
  'Zip',
  'Region',
  'User Name',
  'Discount_Percent'],
 ['Full_Name', 'SSN', 'Credit_Card_Number']]

## Match regex for column header

In [24]:
ssn_check = []
x_check = []

for each_list in column_list:
    for column in each_list: 
        ssn = re.findall("(?i)SSN",column)
        x = re.findall("(?i)^.*name$", column)
        if ssn:
            ssn_check.append(column)
        if x:
            x_check.append(column)
print(ssn_check)
print(x_check)

['SSN', 'SSN', 'SSN']
['full_name', 'NAME', 'First Name', 'Last Name', 'full_name', 'Place Name', 'User Name', 'Full_Name']


In [50]:
ssn_combination = ["ssn","SSN","Ssn","Social Security","Social Security Number", "social_security_number","customer ssn"]

In [57]:
ssn_check=[]
for each_entry in ssn_combination: 
    ssn = re.match("(?i)(^.*(SSN|social|num).*$)",each_entry)
    if ssn:
        ssn_check.append(each_entry)
ssn_check

['ssn',
 'SSN',
 'Ssn',
 'Social Security',
 'Social Security Number',
 'social_security_number',
 'customer ssn']

# email regex

In [63]:
email_combination = ["email","email address","e-mail","e-mail_address","EMAIL","customer_email"]

In [67]:
email_check=[]
for each_entry in email_combination: 
    email = re.match("(?i)(^.*(e.*mail).*$)",each_entry)
    if email:
        email_check.append(each_entry)
email_check

['email',
 'email address',
 'e-mail',
 'e-mail_address',
 'EMAIL',
 'customer_email']

# phone number regex

In [68]:
phone_combination = ["phone_number","number","Phone Number","phone_num","customer_phone_num"]

In [71]:
phone_check=[]
for each_entry in phone_combination: 
    phone = re.match("(?i)(^.*(phone.*num).*$)",each_entry)
    if phone:
        phone_check.append(each_entry)
phone_check

['phone_number', 'Phone Number', 'phone_num', 'customer_phone_num']

# credit card regex

In [81]:
cc_combination = ["credit_card_number","cc_num","cc_number","Credit Card Number","card_num","customer_credit_card"]

In [83]:
cc_check=[]
for each_entry in cc_combination: 
    cc = re.match("(?i)(^.*((credit|cc).*(card|num)).*$)",each_entry)
    if cc:
        cc_check.append(each_entry)
cc_check

['credit_card_number',
 'cc_num',
 'cc_number',
 'Credit Card Number',
 'customer_credit_card']

In [87]:
# address regex

address_check = []
city_check = []
state_check = []
zip_code_check = []
county_check = []

for each_list in column_list:
    for column in each_list: 
        address = re.findall("(?i)^.*address",column)
        city = re.findall("(?i)^city", column)
        county = re.findall("(?i)^.*county",column)
        state = re.findall("(?i)^.*state",column)        
        zip_code = re.findall("(?i)^.*zip",column)  
        county = re.findall("(?i)^.*county",column)        
        if address:
            address_check.append(column)
        if city:
            city_check.append(column)
        if state:
            state_check.append(column)
        if county:
            county_check.append(column)
        if state:
            state_check.append(column)
        if zip_code:
            zip_code_check.append(column)
print(address_check)
print(city_check)
print(state_check)
print(county_check)
print(state_check)
print(zip_code_check)

['ADDRESS', 'ADDRESS']
['CITY', 'CITY', 'City']
['STATE', 'STATE', 'STATE', 'STATE', 'State', 'State']
['COUNTY', 'County']
['STATE', 'STATE', 'STATE', 'STATE', 'State', 'State']
['ZIP', 'ZIP', 'Zip']


# Match regex for data 

# ssn data regex

In [88]:
regex = "^(?!666|000|9\\d{2})\\d{3}-(?!00)\\d{2}-(?!0{4})\\d{4}$"
ssn_data = dummy_data.SSN

In [18]:
ssn_check = []
for each_entry in ssn_data:
    ssn = re.findall(regex,each_entry)
    if ssn:
        ssn_check.append(ssn)
print(ssn_check)

[['489-36-8350'], ['514-14-8905'], ['690-05-5315'], ['421-37-1396'], ['458-02-6124'], ['612-20-6832'], ['300-62-3266'], ['660-03-8360'], ['213-46-8915'], ['449-48-3135'], ['559-81-1301'], ['322-84-2281'], ['646-44-9061'], ['465-73-5022'], ['044-34-6954'], ['477-36-0282'], ['421-90-3440'], ['524-02-7657'], ['205-52-0027'], ['587-03-2682'], ['687-05-8365'], ['751-01-2327'], ['624-84-9181'], ['514-30-2668'], ['451-80-3526'], ['404-12-2154'], ['505-88-5714'], ['461-97-5660'], ['172-32-1176'], ['151-32-2558'], ['489-36-8391'], ['514-14-8946'], ['690-05-5356'], ['421-37-1437'], ['458-02-6165'], ['612-20-6873'], ['300-62-3307'], ['660-03-8401'], ['213-46-8956'], ['449-48-3176'], ['559-81-1342'], ['322-84-2322'], ['646-44-9102'], ['465-73-5063'], ['044-34-6995'], ['477-36-0323'], ['421-90-3481'], ['524-02-7698'], ['205-52-0068'], ['587-03-2723'], ['687-05-8406'], ['751-01-2368'], ['624-84-9222'], ['514-30-2709'], ['451-80-3567'], ['404-12-2195'], ['505-88-5755'], ['461-97-5701'], ['172-32-1217

# Phone number data regex

In [29]:
phone_data= sale_data["Phone No. "]

In [46]:
phone_list = []
for each_entry in phone_data:
    phone = phonenumbers.PhoneNumberMatcher(each_entry, "US")
    if phone:
        phone_list.append(each_entry)

In [47]:
phone_list

['405-959-1129',
 '405-959-1129',
 '405-959-1129',
 '405-959-1129',
 '405-959-1129',
 '405-959-1129',
 '405-959-1129',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '239-335-6755',
 '701-522-6439',
 '701-522-6439',
 '701-522-6439',
 '701-522-6439',
 '701-522-6439',
 '808-380-3891',
 '808-380-3891',
 '808-380-3891',
 '808-380-3891',
 '808-380-3891',
 '808-380-3891',
 '808-380-3891',
 '808-380-3891',
 '808-380-3891',
 '808-380-3891',
 '808-380-3891',
 '307-308-4981',
 '307-308-4981',
 '307-308-4981',
 '307-308-4981',
 '229-364-7348',
 '423-895-1341',
 '423-895-1341',
 '423-895-1341',
 '401-535-8490',
 '401-535-8490',
 '239-462-9760',
 '217-405-0048',
 '217-405-0048',
 '217-405-0048',
 '217-405-0048